In [6]:
from dotenv import load_dotenv
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.language_models.llms import BaseLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
from core import LLMConfigBuilder, llm_retriever

load_dotenv()

True

In [2]:
config = LLMConfigBuilder(
    family="ollama_llms", model="phi3:14b", ls_project_name="meli", llm_args={"temperature": 0}
)

llm = llm_retriever(config)

In [3]:
def invoke_llm_from_prompt(
    llm: BaseLLM | BaseChatModel, prompt: ChatPromptTemplate, input: dict[str, str]
) -> str:
    """This function takes the input and the prompt, creates a chain with
    string parser for the output and returns the output from the LLM.

    Args:
        llm (BaseLLM | BaseChatModel): The LLM to invoke.
        prompt (ChatPromptTemplate): The prompt to use.
        input (dict[str, str]): The input to the LLM based on the prompt.

    Returns:
        str: The output from the LLM.
    """

    chain = prompt | llm | StrOutputParser()

    return chain.invoke(input)


def send_chat_request(llm: BaseChatModel, query: str) -> str:
    response = llm.chat.completions.create(
        messages=[
            {"role": "user", "content": query},
        ],
    )
    chat_result = response.choices[0].message.content
    if not chat_result:
        raise ValueError("No response from the chat model.")
    return chat_result

In [5]:
# Create a chat prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Usted es un sistema que analiza en español el título de una publicación de un producto que están en venta y a partir de dicho título debe determinar si es un producto que se vende por unidad o en paquete o varias unidades. También puede ayudarse de la descripción para categorizar el producto. Please response ONLY with 3 of the following categories: 'multiple_units' if the product is sold in multiple units, 'single_unit' if the product is sold in a single unit, 'package' if the product is sold in a package.",
        ),
        ("user", "Titulo:{title} Descripción:{description}"),
    ]
)

answer = invoke_llm_from_prompt(
    llm,
    prompt,
    {
        "title": "Kit Para Hacer Rosario Completo. Paquete X 3 Unidades + Foll",
        "description": """Cada Paquete Contiene:

- 60 Pepas en Madera de excelente calidad. Medidas de cada pepa: 7 mm
- 1 Crucero Italiano con la imagen de la Virgen Milagrosa.
- 1 Cruz con Medalla de San Benito en metal de excelente calidad.
- 3 Metros de Hilo Terlenka N° 8
- 1 Folleto Plegable de 7 Caras, contienen oraciones, como rezar el rosario, letanías, etc.""",
    },
)
print(answer)

Parent run 0c2bbd9f-454b-412d-8d08-4109586e40ea not found for run 1a7b0f10-7417-4c32-80a3-2ffc2dcbdf93. Treating as a root run.


package


In [10]:
def categorizer(llm, prompt, title, description):
    return invoke_llm_from_prompt(llm, prompt, {"title": title, "description": description})

In [11]:
df = pd.read_csv("/Users/kevinpulido/Github/MeLi/meli_148.csv", sep=";")

df["category"] = df.apply(lambda x: categorizer(llm, prompt, x["title"], x["description"]), axis=1)

Parent run 67dc51e9-ae72-485a-8ecd-37225c38ee8b not found for run 6d390696-9051-4ce3-be50-cbce0963b793. Treating as a root run.
Parent run 3f843748-db8e-4902-8549-3987cdcf683a not found for run 99145858-6292-4858-a589-8c83c57d99b0. Treating as a root run.


,id,title,description,category
0,MCO648996211,Kit Para Hacer Rosario Completo. Paquete X 3 U...,KIT PARA HACER ROSARIO COMPLETO X 3 PAQUETICOS...,multiple_units
1,MCO635227822,Unión 16mm (paquete X 30 Unidades),Sistemas E Irrigaciones te ofrece la Unión 16 ...,paquete


In [ ]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
llm = ChatOllama(model="phi3:14b", temperature=0)
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# using LangChain Expressive Language chain syntax
# learn more about the LCEL on
# /docs/concepts/#langchain-expression-language-lcel
chain = prompt | llm | StrOutputParser()

# for brevity, response is printed in terminal
print(chain.invoke({"topic": "Space travel"}))

In [ ]:
print(llm)

In [ ]:
response = send_chat_request(llm, query="What is Mercado Libre?")
print(response)

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv("/Users/kevinpulido/Github/MeLi/meli_150.csv", sep=";")
df2 = pd.read_csv("/Users/kevinpulido/Github/MeLi/meli.csv", sep=";")

In [ ]:
country_products = df1.merge(df2, on="id", how="left", suffixes=("1", "")).filter(["id", "title"])
country_products.to_csv("/Users/kevinpulido/Github/MeLi/meli_166.csv", sep=";", index=False)

In [ ]:
from core import Country, MercadoLibreItems

In [ ]:
meli = MercadoLibreItems(Country(country="Colombia"))

In [ ]:
country_products["description"] = country_products["id"].apply(
    lambda x: meli.get_product_description(x)["plain_text"]
)
country_products = country_products.drop_duplicates()

In [ ]:
country_products.sample(5)

In [ ]:
country_products.to_csv("/Users/kevinpulido/Github/MeLi/meli_148.csv", sep=";", index=False)